In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer as SIA
from pathlib import Path
from datetime import datetime, timedelta
from gensim.parsing.preprocessing import remove_stopwords
import nltk
import pickle
nltk.download('vader_lexicon')
nltk.download('words')

%matplotlib inline


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gunin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gunin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!

Bad key text.latex.preview in file c:\Users\gunin\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file c:\Users\gunin\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://gith

In [2]:
sia = SIA()
from itertools import chain
import regex as re


def clean_text(input_str: str) -> str:
    input_str = input_str.lower()
    input_str = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", input_str)
    input_str = " ".join(input_str.split())
    input_str = re.sub("[^A-Za-z0-9]"," ",input_str)
    input_str = re.sub(r'^https?:\/\/.*[\r\n]*', '', input_str, flags=re.MULTILINE)
    input_str = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', input_str)
    input_str = re.sub(r"www.\S+",'',input_str)
    input_str = remove_stopwords(input_str)
    input_str = ''.join([i for i in input_str if not i.isdigit()])
    return input_str

def build_sentiment_df(tweet_dataframe, target_column):
    # getting the corresponding data in lists
    raw_tweets = tweet_dataframe[target_column].tolist()
    cleaned_tweets = [clean_text(i) for i in raw_tweets]
    nltk_polarity = [sia.polarity_scores(i)['compound'] for i in cleaned_tweets]
    
    # building the dataframe
    final_df = tweet_dataframe.copy()
    final_df['cleaned_tweets'] = cleaned_tweets
    final_df['nltk_polarity'] = nltk_polarity

    final_df = final_df[final_df['Tweet'].apply(lambda x: len(x.split(',')) < 280)]
    final_df = final_df.drop_duplicates()
    final_df = final_df[final_df['Tweet'].str.contains("rt ") == False]
    final_df = final_df[final_df['nltk_polarity'] != 0.0000] 

    return final_df

In [3]:
def update_tweet_data():
    query = "bitcoin"
    tweets = []
    limit = 60


    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
        # print(vars(tweet))
        # break
        if len(tweets) == limit:
            break
        else:
            tweets.append([tweet.date, tweet.content])
        
    df = pd.DataFrame(tweets, columns=['Date', 'Tweet'])   
    df.reset_index()
    df = df.set_index("Date")
    df = build_sentiment_df(df, "Tweet")
    df = pd.DataFrame(df.resample('1Min').mean()['nltk_polarity'])
    return df

In [4]:
import yfinance as yf

def update_minute_data():
    data = yf.download(tickers='BTC-AUD', period = '3h', interval = '1m')
    data = data.drop(["Open", "High", "Low", "Adj Close", "Volume"], axis = 1)
    data["Actual Returns"] = data["Close"].pct_change()
    data["sma_50"] = data["Close"].rolling(window=50).mean()
    data["sma_100"] = data["Close"].rolling(window=100).mean()
    data = data.dropna()
    return data



In [5]:
def make_data():
    financial_df = update_minute_data()
    tweet_df = update_tweet_data()
    data = pd.concat([financial_df, tweet_df], axis = 1)
    data = data.dropna()

    # failsafe incase no data is generated the first time
    # function will always output data
    if data.shape == (0,5):
        return make_data()
    else:
        return data


In [6]:
with open('model_1', 'rb') as f:
    m1 = pickle.load(f)

with open('model_2', 'rb') as f:
    m2 = pickle.load(f)

with open('model_3', 'rb') as f:
    m3 = pickle.load(f)

with open('eclf', 'rb') as f:
    m4 = pickle.load(f)

with open('svm_model', 'rb') as f:
    m5 = pickle.load(f)

In [7]:
# Imports
from sklearn.preprocessing import StandardScaler


## Discord Bot

In [10]:
import nest_asyncio
import discord
from dotenv import load_dotenv
import os

load_dotenv()

TOKEN = os.getenv("DISCORD_API_TOKEN")
nest_asyncio.apply()


client = discord.Client()

@client.event
async def on_ready():
    print("{0.user} is now online!".format(client))

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    if message.content.startswith('!signal'):
        await message.channel.send('Generating Signal...')

        # Generate data to make prediction on
        data = make_data()
        X = data[["sma_50", "sma_100", 'nltk_polarity']].copy()

        # Create a StandardScaler instance
        scaler = StandardScaler()

        # Apply the scaler model to fit the X-train data
        X_scaler = scaler.fit(X)

        X_scaled = X_scaler.transform(X)

        # get prediciotns from each trained model
        m1_prediction = m1.predict(X_scaled)
        m2_prediction = m2.predict(X_scaled)
        m3_prediction = m3.predict(X_scaled)
        m4_prediction = m4.predict(X_scaled)
        m5_prediction = m5.predict(X_scaled)
        score_lst = [m1_prediction[-1], m2_prediction[-1], m3_prediction[-1], m4_prediction[-1], m5_prediction[-1]]

        if data["Actual Returns"][-1] < 0 and data['nltk_polarity'][-1] > 0.16 or data["Actual Returns"][-1] < 0 and sum(score_lst) >= 3:
            await message.channel.send('Buy')
        elif data["Actual Returns"][-1] > 0 and data['nltk_polarity'][-1] <= 0.3 or data["Actual Returns"][-1] > 0 and sum(score_lst) <= -3:
            await message.channel.send('Sell')
        else:
            await message.channel.send('Hold')
        

client.run(TOKEN)

## Commandline/Basic input output application

In [1]:
# Discord bot code kept giving the above error depite working before 
# this code block serves as an alterative wit the same funtionality

exit = False

def generate_menu():
    print("~~~~~~~~~~MENU~~~~~~~~~~")
    print("[1]     Get Trade Signal")
    print("[2]    Model Predictions")
    print("[3]    Price Information")
    print("[4]    Current Sentiment")
    print("[X]                 Exit")
    print("\nPlease enter digit [1-4] corresponding to what you would like to do or X to exit: ")

while exit == False:
    # prompt
    generate_menu()
    command = input()
    
    if command == '1':
        print("Generating Signals...")
        data = make_data()
        X = data[["sma_50", "sma_100", 'nltk_polarity']].copy()

        # Create a StandardScaler instance
        scaler = StandardScaler()

        # Apply the scaler model to fit the X-train data
        X_scaler = scaler.fit(X)

        X_scaled = X_scaler.transform(X)

        # get prediciotns from each trained model
        m1_prediction = m1.predict(X_scaled)
        m2_prediction = m2.predict(X_scaled)
        m3_prediction = m3.predict(X_scaled)
        m4_prediction = m4.predict(X_scaled)
        m5_prediction = m5.predict(X_scaled)
        score_lst = [m1_prediction[-1], m2_prediction[-1], m3_prediction[-1], m4_prediction[-1], m5_prediction[-1]]

        if (data["Actual Returns"][-1] <= 0 and data['nltk_polarity'][-1] > 0.16) or (data["Actual Returns"][-1] <= 0 and sum(score_lst) >= 3):
            print('Buy')
        elif (data["Actual Returns"][-1] > 0 and data['nltk_polarity'][-1] < 0.16) or (data["Actual Returns"][-1] > 0 and sum(score_lst) <= -3):
            print('Sell')
        elif data["Actual Returns"][-1] > 0 and data['nltk_polarity'][-1] > 0.4:
            print('Hold')
        else:
            print('Hold')
    elif command == 'X':
        exit = True
    # Functionality not added for menu 2-4
        

~~~~~~~~~~MENU~~~~~~~~~~
[1]     Get Trade Signal
[2]    Model Predictions
[3]    Price Information
[4]    Current Sentiment
[X]                 Exit

Please enter digit [1-4] corresponding to what you would like to do or X to exit: 
Generating Signals...


NameError: name 'make_data' is not defined